In [1]:
import numpy as np
import numpy.typing as npt
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib import colors
import gif

In [2]:
def get_neighbours(grid_size: np.int64) -> npt.NDArray[np.int16]:
    L: np.int16 = int(np.sqrt(grid_size))
    neighbours = np.zeros((grid_size, 4), dtype=np.int16)
    for k in np.arange(grid_size):
        neighbours[k, 0] = k + 1
        if (k + 1) % L == 0:
            neighbours[k, 0] = k + 1 - L
        neighbours[k, 1] = k + L
        if k > (grid_size - L - 1):
            neighbours[k, 1] = k + L - grid_size
        neighbours[k, 2] = k - 1
        if k % L == 0:
            neighbours[k, 2] = k + L - 1
        neighbours[k, 3] = k - L
        if k < L:
            neighbours[k, 3] = k + grid_size - L
    return neighbours

In [3]:
# constants

SUSCEPTIBLE = 0
CONTAMINATED = 1
RECOVERED = 2

RED = 1, 0, 0
BLUE = 0, 1, 0
GREEN = 0, 0, 1

COLORS: list[str] = [GREEN, RED, BLUE]
HEALTH_CONDITIONS_IDS: list[str] = ["Susceptible", "Contaminated", "Recovered"]


In [4]:
conditions: list[dict] = [
    {"prob_contamination": 0.9, "prob_recovery": 0.1},
    {"prob_contamination": 0.5, "prob_recovery": 0.5},
    {"prob_contamination": 0.1, "prob_recovery": 0.9}
]

In [5]:
# visualization functions

@gif.frame
def draw_grid(grid: npt.NDArray[np.int8]) -> None:
    cmap = colors.ListedColormap(colors=COLORS)
    bounds = [0, 1, 2]
    grid = np.reshape(grid, newshape=(int(np.sqrt(grid.size)), int(np.sqrt(grid.size))))
    plt.figure(figsize=(8, 4))
    plt.title(f"{grid.shape} grid")
    plt.xticks(np.arange(grid.shape[0] + 1), labels=[])
    plt.yticks(np.arange(grid.shape[0] + 1), labels=[])
    patches = [
        mpatches.Patch(color=COLORS[i], label=f"{HEALTH_CONDITIONS_IDS[i]}")
        for i in np.arange(3)
    ]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.0)
    plt.grid(visible=True, which="both")
    plt.imshow(
        grid, cmap=cmap, origin="lower", extent=(0, grid.shape[0], 0, grid.shape[0])
    )


def create_gif(frames: npt.NDArray[np.int8]) -> None:
    gif.save(frames, "example.gif", duration=200)

In [6]:
def spread(val: np.int8, neighbours_health: npt.NDArray[np.int16], conditions: dict) -> np.int8:
    prob_contamination: np.float32 = conditions["prob_contamination"]
    prob_recovery: np.int32 = conditions["prob_recovery"]
    if val == SUSCEPTIBLE and CONTAMINATED in neighbours_health:
        if np.random.rand() < prob_contamination:
            return CONTAMINATED
        else:
            return val
    elif val == CONTAMINATED:
        if np.random.rand() < prob_recovery:
            return RECOVERED
        else:
            return val
    else:
        return val

def generate_epidemic_process(grid: npt.NDArray[np.int8], conditions: dict, number_of_iterations: np.int8 = 10, gif: bool = False) -> npt.NDArray[np.int8]:  
    neighbours: npt.NDArray[np.int16] = get_neighbours(grid.size)
    frames: list = []
    for _ in np.arange(number_of_iterations):
        if gif:
            frames.append(draw_grid(grid))
        grid = np.array([spread(val=val, neighbours_health=grid[neighbours[idx]], conditions=conditions) for idx, val in enumerate(grid)])
    create_gif(frames)

In [7]:
GRID_SIZE: np.int32 = 1600
grid: npt.NDArray[np.int8] = np.random.choice([0], size=(GRID_SIZE))
grid[324] = 1
grid[651] = 1
generate_epidemic_process(grid=grid, conditions=conditions[1], number_of_iterations=50, gif=True)

# Condições iniciais da rede

De modo a obter uma boa visualização do modelo SIR, uma rede de formato 28x28 foi escolhida, de modo que assim existirão 784 indivíduos na simulação, número mais do que suficiente para analisar o comportamento do modelo para diferentes condições iniciais. Um número de iterações igual a 50 se mostrou um número seguro para garantir a convergência do sistema.

No entanto, tais parâmetros são facilmente manipuláveis e, ao fim das análises principais dos exercícios, alguns testes serão feitos testando as diferentes variações possíveis de condição da rede.

Em relação ao indivíduo contaminado inicial, sua posição foi escolhida aleatoriamente.

# Condições de contaminação e recuperação

As três condições de contaminação e recuperação utilizadas no exercício foram escolhidas com o objetivo de testar o comportamento do modelo para três casos principais, os dois extremos, em que a probabilidade de contaminação de um vizinho é de 90% e de recuperação é de 10%, e a situação simétrica, assim como o caso em que ambas probabilidades são 50%.